# **Face Swaps with Dlib**

####**In this lesson we'll learn:**
1. Perform Face Swapping using Dlib


In [1]:
# Our Setup, Import Libaries, Create our Imshow Function and Download our Images
import cv2
import dlib
import sys
import numpy as np
from matplotlib import pyplot as plt

# Define our imshow function 
def imshow(title = "Image", image = None, size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

In [2]:
# Download and unzip our images and Facial landmark model
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/images.zip
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/shape_predictor_68_face_landmarks.zip
!unzip -qq images.zip
!unzip -qq shape_predictor_68_face_landmarks.zip

--2023-05-01 07:56:46--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/images.zip
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 52.95.142.34
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|52.95.142.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29586716 (28M) [application/zip]
Saving to: ‘images.zip’

images.zip          100%[===================>]  28.22M  18.5MB/s    in 1.5s    

2023-05-01 07:56:48 (18.5 MB/s) - ‘images.zip’ saved [29586716/29586716]

--2023-05-01 07:56:48--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/shape_predictor_68_face_landmarks.zip
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 52.95.142.34
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|52.95.142.34|:443

## **Performing Face Swaps**

In [3]:
# Define Global variables and path to detector
import sys

PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
SCALE_FACTOR = 1 
FEATHER_AMOUNT = 11

FACE_POINTS = list(range(17, 68))
MOUTH_POINTS = list(range(48, 61))
RIGHT_BROW_POINTS = list(range(17, 22))
LEFT_BROW_POINTS = list(range(22, 27))
RIGHT_EYE_POINTS = list(range(36, 42))
LEFT_EYE_POINTS = list(range(42, 48))
NOSE_POINTS = list(range(27, 35))
JAW_POINTS = list(range(0, 17))

In [4]:
# Points used to line up the images.
ALIGN_POINTS = (LEFT_BROW_POINTS + RIGHT_EYE_POINTS + LEFT_EYE_POINTS +
                               RIGHT_BROW_POINTS + NOSE_POINTS + MOUTH_POINTS)

# Points from the second image to overlay on the first. The convex hull of each
# element will be overlaid.
OVERLAY_POINTS = [
    LEFT_EYE_POINTS + RIGHT_EYE_POINTS + LEFT_BROW_POINTS + RIGHT_BROW_POINTS,
    NOSE_POINTS + MOUTH_POINTS,
]

# Amount of blur to use during colour correction, as a fraction of the
# pupillary distance.
COLOUR_CORRECT_BLUR_FRAC = 0.6

# Initialize Detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)

In [5]:
# Detect Facial Landmark
class TooManyFaces(Exception):
  pass

class noFaces(Exception):
  pass

def get_landmarks(img):
  # Return coordinates(x,y)
  ret = detector(img, 1)

  if len(ret) > 1:
    raise TooManyFaces
  if len(ret) == 0:
    raise NoFaces

  matrices = [[p.x, p.y] for p in predictor(img, ret[0]).parts()]
  
  return np.matrix(matrices)

In [6]:
def annotate_landmarks(img, landmarks):
  # Overlay the points from landmark to original image

  img = img.copy()

  # Iterate through the landmarks and draw landmarks in original image
  for idx, point in enumerate(landmarks):
    pos = (point[0, 0], point[0, 1])
    cv2.putText(img, str(idx), pos, fontFace=cv2.FONT_HERSHEY_SCRIPT_COMPLEX, fontScale=0.3, color=(0,255, 0))
    cv2.circle(img, pos, 3, color=(255, 255,0))

  return img

In [ ]:
# Drawconvex used to detect outer points for the landmarks
# More info here: https://docs.opencv.org/3.4/d7/d1d/tutorial_hull.html
def draw_convex_hull(img, points, color):
  points = cv2.convexHull(points)
  cv2.fillConvexPoly(img, points, color=color)